In [1]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('PysparkML').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/25 22:07:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/25 22:07:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Read the dataset

In [6]:
df = spark.read.csv('test3.csv',header=True,inferSchema = True)

In [7]:
df.show()

+------+---+----------+------+
|  name|age|experience|salary|
+------+---+----------+------+
| sunny| 31|         4| 30000|
| krish| 30|         3| 40000|
|  paul| 24|         2| 50000|
|harsha| 32|         4| 45000|
|mahesh| 21|         2| 34000|
|   sai| 23|         2| 30000|
+------+---+----------+------+



In [8]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [9]:
df.columns

['name', 'age', 'experience', 'salary']

In [12]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols = ['age','experience'],outputCol = 'Independent Features')
# VectorAssembler  is used to convert the numerical features into single vector

In [13]:
output = featureassembler.transform(df)
output.show()

+------+---+----------+------+--------------------+
|  name|age|experience|salary|Independent Features|
+------+---+----------+------+--------------------+
| sunny| 31|         4| 30000|          [31.0,4.0]|
| krish| 30|         3| 40000|          [30.0,3.0]|
|  paul| 24|         2| 50000|          [24.0,2.0]|
|harsha| 32|         4| 45000|          [32.0,4.0]|
|mahesh| 21|         2| 34000|          [21.0,2.0]|
|   sai| 23|         2| 30000|          [23.0,2.0]|
+------+---+----------+------+--------------------+



In [16]:
from pyspark.ml.regression import LinearRegression
# trai - test - split
train_data,test_data = output.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol = 'Independent Features',labelCol = 'salary')
regressor = regressor.fit(train_data)

22/12/25 22:28:00 WARN Instrumentation: [9f2f16c4] regParam is zero, which might cause numerical instability and overfitting.
22/12/25 22:28:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/25 22:28:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/25 22:28:00 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [17]:
# coefficients
regressor.coefficients

DenseVector([2631.1475, -14090.1639])

In [18]:
# intercept
regressor.intercept

9450.819672132624

In [19]:
# prediction
pred_results = regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+----+---+----------+------+--------------------+------------------+
|name|age|experience|salary|Independent Features|        prediction|
+----+---+----------+------+--------------------+------------------+
| sai| 23|         2| 30000|          [23.0,2.0]|41786.885245901656|
+----+---+----------+------+--------------------+------------------+



/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [21]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(11786.885245901656, 138930663.80005413)